In [1]:
# -*- coding: utf-8 -*-

import netCDF4 as nc4
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
""" ********  FILES TO PLOT  *************
    *****************************************
    tenseur divrot - freeslip
    convergence 1°/4, /8, /16, /48
    0°, 45°
"""

dirt = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_AM98/"
dirm = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_AM98/MESHMASK/"


ListA = [[4 , 0. , dirt+"EXP_4_een_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n4_t0.nc",60],  
         [8 , 0. , dirt+"EXP_8_een_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n8_t0.nc",60]]

ListB = [[4 , 10. , dirt+"EXP_4_een_fs10_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n4_t10.nc",60],  
         [8 , 10. , dirt+"EXP_8_een_fs10_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n8_t10.nc",60]]

ListC = [[4 , 30. , dirt+"EXP_4_een_fs30_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n4_t30.nc",60],  
         [8 , 30. , dirt+"EXP_8_een_fs30_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n8_t30.nc",60]]

ListD = [[4 , 45. , dirt+"EXP_4_een_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n4_t45.nc",60],  
         [8 , 45. , dirt+"EXP_8_een_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n8_t45.nc",60]]


Listdt = [ListA,ListB,ListC,ListD]

""" *************************** PARAMETERS
"""

NI = 2; NJ = 4
L=2000E3 # 2000 km
vmin = 200 ; vmax = 800
tickszer = [np.float64(i)*L/4. for i in range(5)]
dz=100./3.
N = (vmax-vmin)/dz
cticks = np.arange(200, 800+dz, 100.) # 3 x 100/3
palette = plt.get_cmap('RdBu_r', N)

save = 1 ; psave = "figure12.png" ; dpi = 800


In [4]:
fig, ax = plt.subplots(NI,NJ, figsize=(NJ*3,NI*3), dpi = dpi, squeeze=False)

""" *************************** DATA
"""
for i in range(NI):
    for j in range(NJ):
        n,theta,pdtT,pmm,nmean = Listdt[j][i]
        dx = 100E3/np.float64(n)
        # reading
        print("... dataframe[%d,%d], n = %d" % (i,j,n))
        dt = nc4.Dataset(pdtT)
        mm = nc4.Dataset(pmm)
        tmask = mm.variables['tmask'][0,0]
        glamt = mm.variables['glamt'][0] ; gphit = mm.variables['gphit'][0]
        # grid
        lx = dx*np.cos((45+theta)*np.pi/180)/np.sqrt(2) ; ly = dx*np.sin((45+theta)*np.pi/180)/np.sqrt(2)
        nx,ny = np.shape(glamt)
        gridx = np.zeros((nx,ny)) ; gridy = np.zeros((nx,ny))
        gridx = glamt - lx ; gridy = gphit - ly
        # data
        if nmean>0:
            ssh = dt.variables['sshmeaned'][:,:,:] +500.
            nT,_,_ = np.shape(ssh)
            data = np.mean(ssh[(nT-nmean):,:,:], axis=0)
            data = np.ma.masked_where(tmask==0,data)
        else :
            ssh = dt.variables['sshmeaned'][-1,:,:] +500.
            data = np.ma.masked_where(tmask==0,ssh)
        # plot
        cf = ax[i][j].pcolormesh(gridx, gridy, data,
                            vmin=vmin, vmax=vmax, alpha = 1.,
                            # levels = levels,
                            cmap = palette)
        # ax[i][j].title.set_text(titles[i][j])
        # contour
        c1= ax[i][j].contour(glamt,gphit, data,
                        levels = cticks,
                        vmin=vmin,vmax=vmax,
                        linewidths =0.8, colors=('k',),linestyles = "solid")
        c1= ax[i][j].contour(glamt,gphit, data,
                        levels = [500],
                        vmin=vmin,vmax=vmax,
                        linewidths =1.4, colors=('k',),linestyles = "solid")
        
        
""" *************************** PRETTY
"""
for i in range(NI):
    for j in range(NJ):
        n,t,pdtT,pmm,_ = Listdt[j][i]
        dx = 100E3/4.
        limx = dx ; limy = dx
        # print(tickszer)
        ax[i][j].set_xlim(-limx,L + limx)
        ax[i][j].set_xticks(tickszer)
        ax[i][j].set_ylim(-limy,L + limy)
        ax[i][j].set_yticks(tickszer)
        ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
        #
        #if (i==NI-1):
        #    ax[i][j].set_xticklabels(["%1.1f" % (x/1E6) for x in tickszer])
        #    ax[i][j].set_xlabel("X (1000 km)", fontsize = 6)
        #
        #if (j==0):
        #    ax[i][j].set_yticklabels(["%1.1f" % (x/1E6) for x in tickszer])
        #    ax[i][j].set_ylabel("Y (1000 km)", fontsize = 6)
        #
        ax[i][j].patch.set_color('0.')
        ax[i][j].xaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].yaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].tick_params(axis = "both", which = 'both', width=1., labelsize = 5, pad = 3, \
                             bottom = True, top = True, left = True, right = True)
        ax[i][j].tick_params(which='minor',length = 3)
        ax[i][j].tick_params(which='major',length = 5)
        ax[i][j].set_aspect(aspect='equal') # data coordinate 'equal'

#plt.subplots_adjust(left=0, bottom=0, right=1, top=1, \
#                    wspace=-0.15, hspace=0.1)
plt.subplots_adjust(left=0.01, bottom=0.01, right=0.99, top=0.99, \
                    wspace=0.08, hspace=0.04)

""" *************************** SAVING
"""

if save :
    print("\nsaving : %s" % psave)
    fig.savefig(psave, dpi = dpi)
    plt.close()
    print("figure closed")
plt.show()




... dataframe[0,0], n = 4
... dataframe[0,1], n = 4
... dataframe[0,2], n = 4
... dataframe[0,3], n = 4


<ipython-input-4-83d8a98084d6>:30: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  cf = ax[i][j].pcolormesh(gridx, gridy, data,


... dataframe[1,0], n = 8
... dataframe[1,1], n = 8
... dataframe[1,2], n = 8
... dataframe[1,3], n = 8

saving : figure12.png
figure closed
